In [2]:
'''sizes = [len(rec) for rec in SeqIO.parse("hsp65/hsp65.fasta", "fasta")]
names = [x.description for x in SeqIO.parse("hsp65/hsp65.fasta", "fasta")]
names = [x.split('|')[0] for x in names]
'''

In [34]:
'''with open('hsp65/updated_hsp65.fasta', 'w') as out_file:
    for fasta in SeqIO.parse('hsp65/hsp65.fasta','fasta'):
        SeqIO.write(fasta, out_file, 'fasta')
'''

## The above two chunks are used for sequence formatting

### Create HSP65 Database

In [1]:
import pandas as pd 
import numpy as np 
from Bio import SeqIO
import random
df = pd.read_table('../data/hsp65/nuccore_result_table.txt')
df.head()
random.seed(2018)

In [2]:
### Remove Unverified and unclassified Bacteria
list_names = df['Name']
g = [x.split(' ',2)[0] for x in list_names]
matching = [s == "Mycobacterium" for s in g]
df_filtered = df[matching]
len(df_filtered)

1428

In [3]:
list_names = df_filtered['Name']
g = [x.split(' ',2)[1] for x in list_names]
matching = [s != "sp." for s in g]
df_filtered = df_filtered[matching]
len(df_filtered)

1198

In [4]:
list_names = df_filtered['Name']
g = [x.split(' ',2)[1] for x in list_names]
matching = [s != "massiliense" for s in g]
df_filtered = df_filtered[matching]
len(df_filtered)

1099

In [5]:
### Split String Name
list_names = df_filtered['Name']
g = [x.split(' ',2)[0] for x in list_names]
s = [x.split(' ',2)[1] for x in list_names]
y = list(zip(g,s))
y = [list(x) for x in y]
z = [' '.join(x) for x in y]

In [6]:
df_filtered['Species'] = z
df_filtered.head()

,Name,Length,Accession,GI,Species
0,Mycobacterium timonense strain 5351974 heat,994,EU239792.1,171466780,Mycobacterium timonense
1,Mycobacterium bouchedurhonense strain 4355387 ...,994,EU239790.1,171466776,Mycobacterium bouchedurhonense
2,Mycobacterium marseillense strain 5356591 heat,1462,EU239787.1,171466772,Mycobacterium marseillense
3,Mycobacterium abscessus subsp. bolletii hsp65,401,AB548601.1,315433461,Mycobacterium abscessus
4,Mycobacterium timonense strain 3256799 heat,994,EU239791.1,171466778,Mycobacterium timonense


In [7]:
df2 = df_filtered[['Name','Species']].groupby('Species').agg(['count'])
df2.columns = ['Abundance']
df2.head()

,Abundance
Species,
Mycobacterium abscessus,63
Mycobacterium aemonae,1
Mycobacterium africanum,4
Mycobacterium agri,1
Mycobacterium aichiense,3


### Add Culture Code Regex Boolean

In [8]:
bool1 = df_filtered['Name'].str.contains('ATCC')
bool2 = df_filtered['Name'].str.contains('DSM')
bool3 = df_filtered['Name'].str.contains('CCUG')
bool4 = df_filtered['Name'].str.contains('ASIN')
bool5 = df_filtered['Name'].str.contains('QIA')
bool6 = df_filtered['Name'].str.contains('GTC')

df_filtered['Cultures'] = bool1|bool2|bool3|bool4|bool5|bool6
df_filtered['Cultures'] = df_filtered['Cultures'].astype(int)

In [9]:
df2_s = df2[df2['Abundance'] > 2]
df2_s.head()
test = list(df2_s.index)

## Keep everything in this df
df_auto_pick = df_filtered[~df_filtered['Species'].isin(test)]
small_counts = list(df_auto_pick['Accession'])

### Filter and Randomly Select Species by Culture Code

In [10]:
## Randomly select two species (Keep Cultures)
df_mult = df_filtered[df_filtered['Species'].isin(test)]
len(df_mult)

df3 = df_mult[['Name','Species']].groupby('Species').agg(['count'])
df3.columns = ['Abundance']
df3.head()

,Abundance
Species,
Mycobacterium abscessus,63
Mycobacterium africanum,4
Mycobacterium aichiense,3
Mycobacterium alvei,4
Mycobacterium angelicum,3


In [11]:
## Count Abundance of Culture Codes
df4 = df_mult[['Species', 'Cultures']].groupby('Species').sum()
df4.columns = ['Abundance']
df4.head()

,Abundance
Species,
Mycobacterium abscessus,2
Mycobacterium africanum,2
Mycobacterium aichiense,2
Mycobacterium alvei,0
Mycobacterium angelicum,0


In [12]:
### Multiples with 0
zeros = df4[df4['Abundance'] < 1]
zeros = list(zeros.index)


### Multiples with 1
ones = df4[df4['Abundance'] == 1]
ones = list(ones.index)


### Multiples with 2
twos = df4[df4['Abundance'] == 2]
twos = list(twos.index)

### Multiples with Greater than 2
greater = df4[df4['Abundance'] > 2]
greater = list(greater.index)

In [13]:
## Select Accession of Organism with two cultures 
twos_A = df_mult[df_mult['Species'].isin(twos)]
twos_A = twos_A[twos_A['Cultures'] == 1]
twos_A = list(twos_A['Accession'])
twos_list = twos_A

In [14]:
## Greater than twos
import random
my_acc = []
greatera = df_mult[df_mult['Species'].isin(greater)]
greatera = greatera[greatera['Cultures'] == 1]
for name, group in greatera.groupby('Species'):
    my_list = list(group['Accession'])
    one, two = random.sample(my_list,2)
    my_acc.append(one)
    my_acc.append(two)
   
greater_list = my_acc
len(greater_list)

52

In [15]:
## Ones
my_ones = []
onesa = df_filtered[df_filtered['Species'].isin(ones)]
onesa_1 = onesa[onesa['Cultures'] == 1]
onesa_1 = list(onesa_1['Accession'])
onesa = onesa[onesa['Cultures'] == 0]
for name, group in onesa.groupby('Species'):
    my_list = list(group['Accession'])
    one = random.sample(my_list, 1)
    my_ones.append(one)

my_ones = [item for sublist in my_ones for item in sublist]
ones_list = my_ones+onesa_1

In [16]:
## Zeros
my_zeros = []
zeroa = df_mult[df_mult['Species'].isin(zeros)]
zeroa = zeroa[zeroa['Cultures'] == 0]
for name, group in zeroa.groupby('Species'):
    my_list = list(group['Accession'])
    one, two = random.sample(my_list,2)
    my_zeros.append(one)
    my_zeros.append(two)
    
zeros_list = my_zeros
len(zeros_list)

54

In [17]:
## Combine all Accessions
Combined_Accessions = zeros_list + ones_list + twos_list + greater_list + small_counts

In [18]:
df5 = df_filtered[df_filtered['Accession'].isin(Combined_Accessions)]
len(df5)

319

#### Write Species to Excel and Filter Sequences by Accession

In [19]:
writer = pd.ExcelWriter('Counts_of_Species2.xlsx')
df5.to_excel(writer, "Sheet1")
writer.save()

In [22]:
from Bio import SeqIO

with open('../hsp65.fasta', 'w') as out_file:
    for fasta in SeqIO.parse('../data/hsp65/sequence.fasta','fasta'):
        name, sequence = fasta.id, fasta.seq
        if name in Combined_Accessions:
            SeqIO.write(fasta, out_file, 'fasta')

### Check Accession Overlap Here

In [88]:
from Bio import SeqIO
from collections import defaultdict
records=list(SeqIO.parse("Filtered_hsp65.fasta", "fasta"))

#### Find Records that Share the Same Sequence

In [89]:
from collections import OrderedDict

d = OrderedDict()
for record in records:
    if record.seq in d:
        d[record.seq].append(record)
    else:
        d[record.seq] = [record]
        
len(d)

/Users/stronglab2/anaconda2/lib/python2.7/site-packages/Bio/Seq.py:155: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  "the new string hashing behaviour.", BiopythonWarning)


298

The following chunk createsa new dictionary that contains sequences that have more than one record, then creates a list of true false values to check if the mulitple sequences share the same species

In [90]:
q = {k: v for k, v in d.iteritems() if len(v)> 1}

keep = []

for seq, record_set in q.iteritems():
    
    my_list = []
    for record in record_set:
        
        my_list.append(record.description.split(' ')[2])
    my_list = list(set(my_list))
    if len(my_list) > 1:
        keep.append(True)
    else:
        keep.append(False)

The following chunk creates a dictionary of sequences with the same sequences, but different annotations

In [91]:
weird = dict()
for i in range(len(keep)):
    if keep[i]==True:
        weird[q.keys()[i]] = q.values()[i]
len(weird)

6

In [92]:
for seq, record_set in weird.iteritems():
    print seq + ': (' + str(len(record_set)) + ')'
    for record in record_set:
        print '    ' + record.description

GAGGACCCTTACGAGAAGATCGGCGCCGAGCTGGTCAAGGAAGTCGCCAAGAAGACCGACGACGTTGCCGGTGACGGCACGACGACGGCCACGGTTCTGGCCCAGGCGTTGGTCCGCGAGGGCCTGCGTAACGTCGCCGCCGGCGCCAACCCGCTGGGTCTCAAGCGCGGCATCGAGAAGGCCGTCGAGAAGGTCACCGAGACCCTGCTCAAGTCGGCCAAGGAGGTCGAGACCAAGGACCAGATCGCTGCCACCGCGGCGATTTCGGCGGGCGACCAGTCGATCGGTGACCTCATCGCCGAGGCGATGGACAAGGTCGGCAACGAGGGCGTCATCACCGTCGAGGAGTCCAACACCTTCGGCCTGCAGCTCGAGCTCACCGAGGGCATGCGGTTCGACAAGGGCTACATCTCGGGCTACTTCGTCACCGACGCCGAGCGTCAGGAAGCGGTCCTCGAGGACCCCTTCATCCTGCTGGTCAGCTCCAAGGTGTCGACGGTCAAGGACCTGCTGCCGCTGCTGGAGAAGGTCATCCAGGCCGGCAAGCCGCTGCTGATCATCGCCGAGGACGTCGAGGGTGAGGCTCTGAGCACCCTGGTCGTC: (3)
    KP670342.1 Mycobacterium paraintracellulare strain MOTT64 heat shock protein 65 (hsp65) gene, partial cds
    KP670341.1 Mycobacterium paraintracellulare strain Asan 37128 heat shock protein 65 (hsp65) gene, partial cds
    KF910248.1 Mycobacterium intracellulare strain QIA-9 heat shock protein 65 (hsp65) gene, partial cds
CGCCAAGGAGATCGAGCTGGAGGACCCGTACGAGAAGATCGGCGCCGAGCTGGTCAAGGAAG

In [93]:
count = 0
for seq, record_set in weird.iteritems():
    for record in record_set:
        count += 1
count

15